In [1]:
sealed trait Expr
case class Const(n: Double) extends Expr
case class Plus(e1: Expr, e2: Expr) extends Expr
case class Minus(e1: Expr, e2: Expr) extends Expr
case class Multiply(e1: Expr, e2: Expr) extends Expr
case class Divide(e1: Expr, e2: Expr) extends Expr
case class Ident(id: String) extends Expr
case class Let(id: String, e1: Expr, e2: Expr) extends Expr
case class FunCall(e1: Expr, e2: Expr) extends Expr
case class FunDef(id: String, eBody: Expr) extends Expr

sealed trait Value
case class NumValue(d: Double) extends Value
case object ErrorValue extends Value
case class Closure( id: List[String], e: Expr, definitionTimeEnvironment: Environment ) extends Value

type Environment = Map[String, Value]
val emptyEnv: Environment = Map()

defined trait Expr
defined class Const
defined class Plus
defined class Minus
defined class Multiply
defined class Divide
defined class Ident
defined class Let
defined class FunCall
defined class FunDef
defined trait Value
defined class NumValue
defined object ErrorValue
defined class Closure
defined type Environment
emptyEnv: Environment = Map()

In [2]:
def eval(e: Expr, env: Environment): Value = {
    def hBop(e1: Expr, e2: Expr)(f: (Double, Double) => Double): Value = {
        eval(e1, env) match {
            case NumValue(n1) => eval(e2, env) match {
                case NumValue(n2) => {
                   NumValue(f(n1, n2))
                }
                case _ => ErrorValue
            }
           case _ => ErrorValue
        }
    }
    /*
    This is really the crux of my solution
    This function is called before creating a closure. Param 'e' is the body of the function present in fundef
    Parse goes through the function body and finds any variable name. 
    If e is an Ident, then it is identifying a variable name
    If e is anything else, it calls parse recursively
    FunDef then filters out the variable that is it's parameter, and stores the rest of the list
    This way only things that are necessary for the function to execute get stored. Theoretically
    */
    def parse(e: Expr, acc: List[String]): List[String] = {
        e match {
            case Ident(id) => id +: acc
            case Plus(e1, e2) => parse(e1, acc) ++ parse(e2, acc) ++ acc
            case Minus(e1, e2) => parse(e1, acc) ++ parse(e2, acc) ++ acc
            case Multiply(e1, e2) => parse(e1, acc) ++ parse(e2, acc) ++ acc
            case Divide(e1, e2) => parse(e1, acc) ++ parse(e2, acc) ++ acc
            case FunCall(funName, funBody) => {
                funName match  {
                    case Ident(f) => {
                        List(f) ++ parse(funBody, acc) ++ acc
                    }
                    case FunDef(id, eBody) => {
                        parse(eBody, List()) ++ acc
                    }
                    case _ => {
                        println("you forgot a case in the FunCall case of parse :skull:")
                        acc
                    }
                }
            }
            case Let(id, e1, e2) => ???
            /*
            I think the we need to filter through e2 to find any variables that aren't id and return those. 
            */
            case _ => ???
        }
    }

    e match {
        case Const(n) => NumValue(n)
        case Plus(e1, e2) => hBop(e1, e2) (_ + _)
        case Minus(e1, e2) => hBop(e1, e2) (_ - _)
        case Multiply(e1, e2) => hBop(e1, e2) (_ * _)
        case Divide(e1, e2) => hBop(e1, e2) (_ / _)
         case Ident(x) => env get x match {
            case None => ErrorValue
            case Some(v) => v
        }
        case Let(id, eBind, eRest) => eval(eBind, env) match {
            case v1 => eval(eRest, env + { id -> v1 })
            case _ => ErrorValue
        }
        case FunDef(id, e) => {
            println("Expr e in FunDef Case", e)
            //get the list of variable names from parse and filter out the parameter
            val open_variables = parse(e, List()).filter({
                case variable_name => variable_name != id
            })
            println("List of open variables in FunDef case", open_variables)
            //attach the open variables found above to their respective value
            val new_pairs:List[(String, Value)] = open_variables.map({
                case variable_name => {
                    env get variable_name match {
                        case None => {
                            println("Failed to find variable name in map!")
                            ("error!", ErrorValue)
                        }
                        //assuming the variable exists, attach it to its value and add it to the list
                        case Some(v) => (variable_name, v)
                    }
                }
            })
            //instead of the environment, use the list we generated to store only the relevant variable names
            Closure(List(id), e, new_pairs.toMap)
        }
        case FunCall(eWantFunc, eArg) => eval(eWantFunc, env) match {
            case Closure( param +: List(), eBody, envAtDefinition ) => eval(eArg, env) match {
                case vArg => {
                    println("envAtDefinition in Closure in FunCall Case", envAtDefinition)
                    eval(eBody, envAtDefinition + ( param -> vArg ) )
                }
            }
            case _ => ErrorValue
        }
        case _ => ???
    }
}

def evaluate(e: Expr): Value = eval(e, emptyEnv)

assert(evaluate(Const(5)) == NumValue(5))
assert(evaluate(Plus(Const(5), Const(6))) == NumValue(11))
assert(evaluate(Plus(Plus(Const(1), Const(2)), Plus(Const(3), Const(4)))) == NumValue(10))
/*
Let trash  = 6 in
    let x = 5 in
        let f = function(y) y * x in
            let y = f(3) in
                y + trash
21
*/
val test1 = Let("trash", Const(6), 
               Let("x", Const(5), 
                   Let("f", FunDef("y", Multiply(Ident("y"), Ident("x"))),
                       Let("y", FunCall(Ident("f"), Const(3)),
                           Plus(Ident("y"),Ident("trash"))))))
                           
assert(evaluate(test1) == NumValue(21))

/*
let trash = 6 in
let x = 1 in
let f = function(y) x + y in
let z = 2 in
let g = function(y) f(y) + z in
g(7) + trash
testing multiple functions
needed to add more cases to parse, i only had multiply lol
*/
println("=====STARTING TEST 2=====")
val test2 = Let("trash", Const(6),
                Let("x", Const(1),
                Let("f", FunDef("y", Plus(Ident("x"), Ident("y"))),
                    Let("z", Const(2),
                        Let("g", FunDef("y", Plus(FunCall(Ident("f"), Ident("y")), Ident("z"))),
                            Plus(FunCall(Ident("g"), Const(7)), Ident("trash")))))))
assert(evaluate(test2) == NumValue(16))

/*
let trash = 6 in
let x = 1 in
let f = function(y) x + y in
let z = 2 in
let g = function(y) f(y) + z in
g(7) + trash
rewriting the Expr to have FunDef instead of Ident for the first Expr of FunCall so I can visualize what to do for that case in Parse
I think this is right
*/
println("=====STARTING TEST 3=====")
val fDef3 = FunDef("y", Plus(Ident("x"), Ident("y")))
val test3 = Let("trash", Const(6),
                Let("x", Const(1),
                    Let("z", Const(2),
                        Let("g", FunDef("y", Plus(FunCall(fDef3, Ident("y")), Ident("z"))),
                            Plus(FunCall(Ident("g"), Const(7)), Ident("trash"))))))
assert(evaluate(test3) == NumValue(16))

/*
let trash = 2 in
let f = function(x) let trash = 3 in
    x + trash in
f(1) + trash
*/

val fDef4 = FunDef("x", Let("trash", Const(3), 
                            Plus(Ident("x"), Ident("trash"))))
val test4 = Let("trash", Const(2), 
                Plus(FunCall(fDef4, Const(1)), Ident("trash")))
assert(evaluate(test4) == NumValue(6))
/*
TO EXPLORE IN FURTHER TESTS
Function bodies can have let and ident in them. Need to think about how parse handles that
My intuition is telling me there's an edge case with shadowed variables that breaks my code, but I can't pin down what it is.
The more I think about this the more I think I'm wrong. My gut was that like if a variable in the funciton had the same name as a variable outside the
function there would be a problem. I don't think there is, actually, since I'm doing static scope. I think since the closure only stores things that have
been picked out by parse, there's no issue with an outside variable with the same name as the parameter sneaking into the enviroment, but i'm not 100% sure
Is this different for multi-function calls?
That might be all the cases (for this relatively simply example)
from there it's efficiency and refactoring
*/

(Expr e in FunDef Case,Multiply(Ident(y),Ident(x)))
(List of open variables in FunDef case,List(x))
(envAtDefinition in Closure in FunCall Case,Map(x -> NumValue(5.0)))
=====STARTING TEST 2=====
(Expr e in FunDef Case,Plus(Ident(x),Ident(y)))
(List of open variables in FunDef case,List(x))
(Expr e in FunDef Case,Plus(FunCall(Ident(f),Ident(y)),Ident(z)))
(List of open variables in FunDef case,List(f, z))
(envAtDefinition in Closure in FunCall Case,Map(f -> Closure(List(y),Plus(Ident(x),Ident(y)),Map(x -> NumValue(1.0))), z -> NumValue(2.0)))
(envAtDefinition in Closure in FunCall Case,Map(x -> NumValue(1.0)))
=====STARTING TEST 3=====
(Expr e in FunDef Case,Plus(FunCall(FunDef(y,Plus(Ident(x),Ident(y))),Ident(y)),Ident(z)))
(List of open variables in FunDef case,List(x, z))
(envAtDefinition in Closure in FunCall Case,Map(x -> NumValue(1.0), z -> NumValue(2.0)))
(Expr e in FunDef Case,Plus(Ident(x),Ident(y)))
(List of open variables in FunDef case,List(x))
(envAtDefinition in Closure in

: 